# Federated learning: Aggregation operators

In this notebook we provide an explanation of the implementation of the different federated aggregation operators provided in the platform. Before discussing the different aggregation operators, we establish the federated configuration (for more information see [Basic Concepts Notebook](./basic_concepts.ipynb)).

In [ ]:
import matplotlib.pyplot as plt
import shfl
import keras
import numpy as np


database = shfl.data_base.Emnist()
train_data, train_labels, val_data, val_labels, test_data, test_labels = database.load_data()

iid_distribution = shfl.data_distribution.IidDataDistribution(database)
federated_data, test_data, test_labels = iid_distribution.get_federated_data(num_nodes=20, percent=10)

def model_builder():
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', strides=1, input_shape=(28, 28, 1)))
    model.add(keras.layers.MaxPooling2D(pool_size=2, strides=2, padding='valid'))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', strides=1))
    model.add(keras.layers.MaxPooling2D(pool_size=2, strides=2, padding='valid'))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dropout(0.1))
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dense(10, activation='softmax'))

    model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
    
    return shfl.model.DeepLearningModel(model)


class Reshape(shfl.private.FederatedTransformation):
    
    def apply(self, labeled_data):
        labeled_data.data = np.reshape(labeled_data.data, (labeled_data.data.shape[0], labeled_data.data.shape[1], labeled_data.data.shape[2],1))
        
shfl.private.federated_operation.apply_federated_transformation(federated_data, Reshape())

class Normalize(shfl.private.FederatedTransformation):
    
    def __init__(self, mean, std):
        self.__mean = mean
        self.__std = std
    
    def apply(self, labeled_data):
        labeled_data.data = (labeled_data.data - self.__mean)/self.__std
        
        
mean = np.mean(train_data.data)
std = np.std(train_data.data)
shfl.private.federated_operation.apply_federated_transformation(federated_data, Normalize(mean, std))

test_data = np.reshape(test_data, (test_data.shape[0], test_data.shape[1], test_data.shape[2],1))

Once we have loaded and federated the data and established the learning model, it only remains to establish the aggregation operator. At the moment, the following classes are included in the platform: `FedAvg` and `WeightedFedAvg`. The implementations of the federated aggregation operators are as follows.

## Federated Averaging (`FedAvg`) Operator

In this section, we detail the implementation of `FedAvg` (see [FedAVg](../shfl/federated_aggregator/avgfed_aggregator.py)) proposed  by Google in this [paper](https://arxiv.org/abs/1602.05629). 

It is based on the arithmetic mean of the local parameters $W_i$ trained in each of the local clients $C_i$. That is, the parameters $W$ of the global model after each round of training are

$$W = \frac{1}{n_{\rm{C}}} \sum_{i=1}^{n_{\rm{C}}} W_i$$


For its implementation, we create a class that implements [FederatedAggregator](../shfl/federated_aggregator/federated_aggregator.py) interface. The method `aggregate_weights` is overwritten calculating the mean of the local weights of each client.

In [ ]:
import numpy as np

from shfl.federated_aggregator.federated_aggregator import FederatedAggregator


class AvgFedAggregator(FederatedAggregator):
    """
    Implementation of Average Federated Aggregator. It only uses a simple average of the parameters of all the models
    """

    def aggregate_weights(self, clients_params):
        clients_params_array = np.array(clients_params)

        num_clients = clients_params_array.shape[0]
        num_layers = clients_params_array.shape[1]
        clients_params_array = clients_params_array.reshape(num_clients, num_layers)

        aggregated_weights = np.array([np.mean(clients_params_array[:, layer], axis=0) for layer in range(num_layers)])

        return aggregated_weights


fedavg_aggregator = AvgFedAggregator()

## Weighted Federated Averaging (`WeightedFedAvg`) Operator

In this section, we detail the implementation of `WeightedFedAvg` (see [WeightedFedAVg](../shfl/federated_aggregator/weighted_avgfed_aggregator.py)). It is the weighted version of `FedAvg`. The weight of each client $C_i$ is determined by the amount of client's data $n_i$ with respect to total training data $n$. That is, the parameters $W$ of the global model after each round of training are 

$$W =  \sum_{i=1}^n \frac{n_i}{n} W_i$$

When all clients have the same amount of data it is equivalent to FedAvg.

For its implementation, we create a class that implements `FederatedAggregator` interface. The method `aggregate_weights` is overwritten calculating the weighted mean of the local parameters of each client. For that purpose, we first ponderate the local parameters by the percentage and, after that, we sum the ponderated parameters.

In [ ]:
import numpy as np

from shfl.federated_aggregator.federated_aggregator import FederatedAggregator


class WeightedAvgFedAggregator(FederatedAggregator):
    """
    Implementation of Average Federated Aggregator. The aggregation of the parameters is based in the number of data \
    in every node.
    """

    def aggregate_weights(self, clients_params):
        clients_params_array = np.array(clients_params)

        num_clients = clients_params_array.shape[0]
        num_layers = clients_params_array.shape[1]
        clients_params_array = clients_params_array.reshape(num_clients, num_layers)

        ponderated_weights = np.array([self._percentage[client] * clients_params_array[client, :] for client in range(num_clients)])
        aggregated_weights = np.array([np.sum(ponderated_weights[:, layer], axis=0) for layer in range(num_layers)])

        return aggregated_weights
    
weighted_fedavg_aggregator = WeightedAvgFedAggregator()

Finally, we are ready to establish de federated goverment with any of the implemented aggregation operators and start the federated learning process.
The class `FederatedGovernment` manages the deployment of the model over the federated data, and is in charge of aggregating the parameters into a central node.

In [ ]:
federated_government = shfl.learning_approach.FederatedGovernment(model_builder, federated_data, fedavg_aggregator)

In [ ]:
federated_government.run_rounds(1, test_data, test_labels)